# Importing libraries

In [90]:
import numpy as np
import pandas as pd

In [91]:
from sklearn.compose import ColumnTransformer

In [92]:
from sklearn.preprocessing import OneHotEncoder

In [110]:
from sklearn.model_selection import train_test_split

In [116]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [118]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.linear_model import LinearRegression, Ridge,Lasso

# Data Preprocessing

In [93]:
df = pd.read_csv('D:/mlproject/notebook/data/stud.csv')

In [94]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


We will predict the average scores of students

In [95]:
df['average_score'] = (df['math_score'] + df['reading_score'] + df['writing_score'])/3

In [96]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score,average_score
0,female,group B,bachelor's degree,standard,none,72,72,74,72.666667
1,female,group C,some college,standard,completed,69,90,88,82.333333
2,female,group B,master's degree,standard,none,90,95,93,92.666667
3,male,group A,associate's degree,free/reduced,none,47,57,44,49.333333
4,male,group C,some college,standard,none,76,78,75,76.333333


In [97]:
X = df.drop(['math_score', 'reading_score', 'writing_score', 'average_score'], axis=1)
y = df['average_score']

We will map all the 'some high school' value to 'high school' in parental_level_of_education feature

In [98]:
df['parental_level_of_education'].unique()

array(["bachelor's degree", 'some college', "master's degree",
       "associate's degree", 'high school', 'some high school'],
      dtype=object)

In [99]:
df['parental_level_of_education'] = np.where(df['parental_level_of_education'] == 'some high school',
         'high school',
         df['parental_level_of_education'])

In [100]:
X.columns

Index(['gender', 'race_ethnicity', 'parental_level_of_education', 'lunch',
       'test_preparation_course'],
      dtype='object')

In [101]:
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ('One Hot Encoder', oh_transformer, list(X.columns))
    ]
)

In [102]:
X = preprocessor.fit_transform(X).toarray()

In [104]:
X

array([[1., 0., 0., ..., 1., 0., 1.],
       [1., 0., 0., ..., 1., 1., 0.],
       [1., 0., 0., ..., 1., 0., 1.],
       ...,
       [1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 1., 1., 0.],
       [1., 0., 0., ..., 0., 0., 1.]])

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [113]:
X_train.shape

(800, 17)

In [114]:
X_test.shape

(200, 17)

In [117]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2 = r2_score(true, predicted)
    return mae, mse, rmse, r2

In [120]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "AdaBoost Regressor": AdaBoostRegressor()
}

model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_mse, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_mse, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Mean Squared Error: {:.4f}".format(model_train_mse))
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Mean Squared Error: {:.4f}".format(model_test_mse))
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('\n')

Linear Regression
Model performance for Training set
- Mean Squared Error: 149.2249
- Root Mean Squared Error: 12.2158
- Mean Absolute Error: 9.9658
- R2 Score: 0.2518
----------------------------------
Model performance for Test set
- Mean Squared Error: 182.2909
- Root Mean Squared Error: 13.5015
- Mean Absolute Error: 10.5992
- R2 Score: 0.1496


Lasso
Model performance for Training set
- Mean Squared Error: 174.1461
- Root Mean Squared Error: 13.1964
- Mean Absolute Error: 10.5625
- R2 Score: 0.1269
----------------------------------
Model performance for Test set
- Mean Squared Error: 199.7639
- Root Mean Squared Error: 14.1338
- Mean Absolute Error: 11.0886
- R2 Score: 0.0681


Ridge
Model performance for Training set
- Mean Squared Error: 148.7352
- Root Mean Squared Error: 12.1957
- Mean Absolute Error: 9.9355
- R2 Score: 0.2543
----------------------------------
Model performance for Test set
- Mean Squared Error: 179.6503
- Root Mean Squared Error: 13.4034
- Mean Absolute Err